In [ ]:
from requests import get # la vamos a usar para solicitar la página mediante su URL 
from bs4 import BeautifulSoup # biblioteca para web scraping
import pandas as pd
import string                              
from nltk.corpus import stopwords # para remover palabras que no agregan significado          
from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
from IPython.display import clear_output
from collections import defaultdict
import numpy as np

In [1]:
def etiquetas(palabras,frecuencias,clases):
    import numpy as np
    x = np.zeros((1, 6))
    for palabra in palabras:
        for t in clases:
            x[0,t] += frecuencias.get((palabra, t),0)
    x[0,0]=1
    return x

stop        = stopwords.words('english')
tokenizer   = TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)
stemmer     = PorterStemmer()
webs        = pd.ExcelFile("WEBs.xls")
df          = webs.parse("Categorias")
textos      = pd.DataFrame(columns = [ 'Texto' , 'Clase'])
clases_df   = set(df["CÓDIGO"].value_counts().index)
frecuencias = defaultdict(int)
for i in df.index:
    url       = df["URL"][i]
    pagina    = get(url)
    html      = BeautifulSoup(pagina.text,"html.parser")
    parrafos  = [ x.text.strip() for x in html.findAll("p") if x.text != "\n" ]
    for p in parrafos:
        palabras = p.split()
        if 15 <= len(palabras):
            clear_output( wait = True )
            print("procesando {}".format(url))
            print(f"{i} de {len(df)}")
            texto_tokens = tokenizer.tokenize(p)
            texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation ]
            texto_stem   = [ stemmer.stem(word) for word in texto_clean ]
            for t in range(len(texto_stem)//20):
                j = 20 * t
                palabras  = texto_stem[ j : 20 * (t+1) ] 
                textos   = textos.append({'Texto' : palabras, 'Clase' : df["CÓDIGO"][i] }, ignore_index = True)

vocabulario = set([x for m in textos.index for x in textos.loc[m].Texto ])                
clases_df   = set(df["CÓDIGO"].value_counts().index)

for i in textos.index:
    for palabra in textos.loc[i].Texto:
        frecuencias[(palabra, textos.loc[i].Clase)] += 1
        clear_output( wait = True )
        print(f"Creando Frecuencias {i} de {len(textos)}")

for i in clases_df:
    textos[f"Clase{i}"] = textos.Texto.apply(lambda x : etiquetas(x,frecuencias,clases_df)[0,i])
print("dataset terminado")
print(textos.describe())

Creando Frecuencias 15007 de 15008
dataset terminado
             Clase1        Clase2        Clase3        Clase4        Clase5
count  15008.000000  15008.000000  15008.000000  15008.000000  15008.000000
mean     761.385328    821.676639    778.788380    707.042844    748.152119
std      398.363752    386.528678    341.214019    470.108328    578.907203
min       11.000000      8.000000      0.000000      6.000000      1.000000
25%      494.000000    544.000000    532.000000    391.000000    352.000000
50%      682.000000    759.000000    732.000000    572.000000    585.000000
75%      929.000000   1027.000000    972.000000    869.250000    946.000000
max     5122.000000   3158.000000   2793.000000   4928.000000   4387.000000


In [3]:
textos

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
0,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",1,1266.0,468.0,457.0,317.0,462.0
1,"[bc, commonli, known, alexand, great, greek, ὁ...",1,2073.0,1007.0,737.0,279.0,522.0
2,"[pella, 356, bc, succeed, father, philip, ii, ...",1,1457.0,1144.0,1205.0,698.0,1348.0
3,"[age, thirti, creat, one, largest, empir, anci...",1,1272.0,1307.0,1022.0,829.0,1088.0
4,"[youth, alexand, tutor, aristotl, age, 16, phi...",1,1497.0,694.0,620.0,485.0,579.0
...,...,...,...,...,...,...,...
15003,"[south, chile, mont, verd, citat, need, americ...",5,532.0,528.0,555.0,618.0,1112.0
15004,"[common, style, stone, tool, product, make, kn...",5,701.0,650.0,597.0,370.0,2110.0
15005,"[archaic, period, america, saw, chang, environ...",5,682.0,782.0,740.0,593.0,1405.0
15006,"[mobil, hunter-gather, individu, group, start,...",5,641.0,560.0,526.0,572.0,1144.0


In [4]:
textos.describe()

,Clase1,Clase2,Clase3,Clase4,Clase5
count,15008.000000,15008.000000,15008.000000,15008.000000,15008.000000
mean,761.385328,821.676639,778.788380,707.042844,748.152119
std,398.363752,386.528678,341.214019,470.108328,578.907203
min,11.000000,8.000000,0.000000,6.000000,1.000000
25%,494.000000,544.000000,532.000000,391.000000,352.000000
50%,682.000000,759.000000,732.000000,572.000000,585.000000
75%,929.000000,1027.000000,972.000000,869.250000,946.000000
max,5122.000000,3158.000000,2793.000000,4928.000000,4387.000000


In [5]:
textos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15008 entries, 0 to 15007
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Texto   15008 non-null  object 
 1   Clase   15008 non-null  object 
 2   Clase1  15008 non-null  float64
 3   Clase2  15008 non-null  float64
 4   Clase3  15008 non-null  float64
 5   Clase4  15008 non-null  float64
 6   Clase5  15008 non-null  float64
dtypes: float64(5), object(2)
memory usage: 820.9+ KB


In [6]:
textos.to_csv("Historia.csv")

In [7]:
import pandas as pd
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
0,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",1,1266.0,468.0,457.0,317.0,462.0
1,"['bc', 'commonli', 'known', 'alexand', 'great'...",1,2073.0,1007.0,737.0,279.0,522.0
2,"['pella', '356', 'bc', 'succeed', 'father', 'p...",1,1457.0,1144.0,1205.0,698.0,1348.0
3,"['age', 'thirti', 'creat', 'one', 'largest', '...",1,1272.0,1307.0,1022.0,829.0,1088.0
4,"['youth', 'alexand', 'tutor', 'aristotl', 'age...",1,1497.0,694.0,620.0,485.0,579.0
...,...,...,...,...,...,...,...
15003,"['south', 'chile', 'mont', 'verd', 'citat', 'n...",5,532.0,528.0,555.0,618.0,1112.0
15004,"['common', 'style', 'stone', 'tool', 'product'...",5,701.0,650.0,597.0,370.0,2110.0
15005,"['archaic', 'period', 'america', 'saw', 'chang...",5,682.0,782.0,740.0,593.0,1405.0
15006,"['mobil', 'hunter-gather', 'individu', 'group'...",5,641.0,560.0,526.0,572.0,1144.0


In [8]:
textos.groupby(by=["Clase"]).count()

,Texto,Clase1,Clase2,Clase3,Clase4,Clase5
Clase,,,,,,
1,2988,2988,2988,2988,2988,2988
2,3166,3166,3166,3166,3166,3166
3,3201,3201,3201,3201,3201,3201
4,2729,2729,2729,2729,2729,2729
5,2924,2924,2924,2924,2924,2924


In [9]:
textos.sample(10)

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
6721,"['radiu', 'vector', 'see', ""newton'"", 'law', '...",3,497.0,657.0,903.0,524.0,597.0
13944,"['african', 'contin', 'three', 'area', 'identi...",5,619.0,512.0,500.0,445.0,1130.0
11315,"['pacif', 'us', 'forc', 'continu', 'press', 'b...",4,777.0,864.0,912.0,1589.0,271.0
10853,"['war', 'repar', 'direct', 'cost', 'war', 'aus...",4,753.0,671.0,1053.0,2275.0,116.0
4764,"['william', 'henri', 'includ', 'stori', 'quarr...",2,564.0,1176.0,1114.0,489.0,573.0
9919,"['increas', 'irrelev', 'holi', 'allianc', '182...",4,399.0,867.0,1035.0,1053.0,421.0
14108,"['collect', 'call', 'projectil', 'point', 'pro...",5,646.0,614.0,606.0,506.0,978.0
1856,"['patria', 'potesta', 'even', 'extend', 'adult...",1,775.0,870.0,667.0,391.0,946.0
983,"['aspect', ""carthage'"", 'polit', 'system', 'pe...",1,2151.0,1244.0,716.0,631.0,837.0
9316,"['greater', 'abil', 'resuppli', 'coloni', 'pro...",3,954.0,1100.0,1398.0,1505.0,620.0


In [20]:
len(vocabulario)

25087

In [21]:
len(frecuencias)

44829

In [12]:
clases_df

{1, 2, 3, 4, 5}

In [13]:
import pickle
def save_obj(obj, name):
       with open(name + '.pkl', 'wb') as f:
           pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
       with open(name + '.pkl', 'rb') as f:
           return pickle.load(f)

In [14]:
save_obj(vocabulario, "Vocabulario")

In [15]:
save_obj(frecuencias, "Frecuencias")

In [19]:
save_obj(clases_df, "clases_df")

In [18]:
textos.sample(10)

,Texto,Clase,Clase1,Clase2,Clase3,Clase4,Clase5
5937,"['κρατοῦσα', 'διάλεκτος', 'hē', 'kratousa', 'd...",2,1220.0,1304.0,663.0,521.0,489.0
548,"['statu', 'stand', 'guard', 'citi', 'raze', 'g...",1,1029.0,691.0,598.0,525.0,595.0
5191,"['research', 'suggest', 'vike', 'may', 'origin...",2,599.0,948.0,637.0,433.0,863.0
8812,"['throughout', 'south', 'american', 'hemispher...",3,752.0,797.0,907.0,798.0,1225.0
12000,"['new', 'order', 'arriv', 'radic', 'kronstadt'...",4,1167.0,1171.0,1376.0,1170.0,1125.0
4574,"['launch', 'norman', 'conquest', 'england', 's...",2,656.0,1564.0,1042.0,445.0,745.0
440,"['gold', 'need', 'crush', 'probabl', 'use', 'c...",1,1014.0,767.0,808.0,516.0,1113.0
2883,"['19th', '20th', 'centuri', 'scholar', 'common...",1,1605.0,1362.0,698.0,365.0,1396.0
10543,"['day', 'sent', 'belgian', 'govern', 'ultimatu...",4,731.0,1027.0,932.0,1250.0,542.0
1983,"['late', '4th', 'millennium', 'bc', 'sumer', '...",1,1161.0,454.0,367.0,391.0,845.0
